In [1]:
import requests
import json
import pandas as pd
import psycopg2

In [2]:
# Creando la conexión a Redshift

url="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
data_base="data-engineer-database"
user="diegoalonsotelloalva1998_coderhouse"
pwd= 'KTs5N49gGd'

try:
    conn = psycopg2.connect(
        host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
        dbname=data_base,
        user=user,
        password=pwd,
        port='5439'
    )
    print("Conectado a Redshift con éxito!")
    
except Exception as e:
    print("No es posible conectar a Redshift")

Conectado a Redshift con éxito!


In [3]:
#Crear la tabla si no existe
with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS diegoalonsotelloalva1998_coderhouse.bitcoins
        (
         compra_bitcon_pesoarg FLOAT
        ,comision_bitcon_pesoarg FLOAT 
        ,venta_bitcon_pesoarg FLOAT 
        ,compra_trxusdt_pesoarg FLOAT
        ,comision_trxusdt_pesoarg FLOAT
        ,venta_trxusdt_pesoarg FLOAT
        ,fecha_actualizacion VARCHAR(255)
        ,comision_retiro FLOAT
        )
    """)
    conn.commit()

In [4]:
#Vaciar la tabla para evitar duplicados o inconsistencias
with conn.cursor() as cur:
  cur.execute("Truncate table bitcoins")
  count = cur.rowcount

In [5]:
#consultando la tabla
cur = conn.cursor()
cur.execute("SELECT * FROM bitcoins")
results = cur.fetchall()
results

[]

In [6]:
# La API devuelve los últimos precios de compra y venta de los pares BTC / ARS, TRX-USDT / ARS y comisiones.

#Paso 1 Extraer la data

data = []

response = requests.get('https://www.bitmonedero.com/api/btc-ars')
data_json = json.loads(response.text)
data.append(data_json)
  
#Paso 2 Crear el diccionario

diccionario = {'compra_bitcon_pesoarg':[],'comision_bitcon_pesoarg':[],'venta_bitcon_pesoarg':[],'compra_trxusdt_pesoarg':[]
               ,'comision_trxusdt_pesoarg':[],'venta_trxusdt_pesoarg':[],'fecha_actualizacion':[],'comision_retiro':[]}

# compra_bitcon_pesoarg
# comision_bitcon_pesoarg
# venta_bitcon_pesoarg
# compra_trxusdt_pesoarg
# comision_trxusdt_pesoarg
# venta_trxusdt_pesoarg
# fecha_actualizacion
# comision_retiro

#Extraer cada campo de la data
        
for i in data:
    compra_bitcon_pesoarg = i['buy_btc_ars']
    comision_bitcon_pesoarg= i['buy_btc_ars_fee']
    venta_bitcon_pesoarg= i['sell_btc_ars']
    compra_trxusdt_pesoarg= i['buy_trxusdt_ars']
    comision_trxusdt_pesoarg = i['buy_trxusdt_ars_fee']
    venta_trxusdt_pesoarg= i['sell_trxusdt_ars']
    fecha_actualizacion= i['updated_at_prices']
    comision_retiro= i['withdrawal_fee']
    
    diccionario['compra_bitcon_pesoarg'].append(compra_bitcon_pesoarg)
    diccionario['comision_bitcon_pesoarg'].append(comision_bitcon_pesoarg)
    diccionario['venta_bitcon_pesoarg'].append(venta_bitcon_pesoarg)
    diccionario['compra_trxusdt_pesoarg'].append(compra_trxusdt_pesoarg)
    diccionario['comision_trxusdt_pesoarg'].append(comision_trxusdt_pesoarg)
    diccionario['venta_trxusdt_pesoarg'].append(venta_trxusdt_pesoarg)
    diccionario['fecha_actualizacion'].append(fecha_actualizacion)
    diccionario['comision_retiro'].append(comision_retiro)
    
df = pd.DataFrame(diccionario)
display(df)

,compra_bitcon_pesoarg,comision_bitcon_pesoarg,venta_bitcon_pesoarg,compra_trxusdt_pesoarg,comision_trxusdt_pesoarg,venta_trxusdt_pesoarg,fecha_actualizacion,comision_retiro
0,74094065.85,11114.109877,6.805107e+07,1088.514094,1088.514094,999.736527,2024-03-18T03:52:37.000000Z,0.045


In [8]:
#Insertando los datos en Redsfhift
from psycopg2.extras import execute_values
with conn.cursor() as cur:
    execute_values(
        cur,
        '''
        INSERT INTO bitcoins (compra_bitcon_pesoarg,comision_bitcon_pesoarg,venta_bitcon_pesoarg,compra_trxusdt_pesoarg
        ,comision_trxusdt_pesoarg,venta_trxusdt_pesoarg,fecha_actualizacion,comision_retiro)
        VALUES %s
        ''',
        [tuple(row) for row in df.values],
        page_size=len(df)
    )
    conn.commit()

In [9]:
cur.close()
conn.close()